In [53]:
import uproot, uproot_methods
import uproot_methods.classes.TLorentzVector as TLorentzVector
import matplotlib.pyplot as plt
import numpy as np

t = 0.7  # transparency of plots

testRun = False
dR_cut_quarkToJet = 0.05

In [2]:
def returnListOfTruthBQuarkIndicesByDaughters(a_D1, a_D2, a_PID):
    bQuarkIndices = []
    
    for iParticle in range(0, len(a_D1)):
         if a_PID[iParticle]==25:
            daughter1 = a_D1[iParticle]
            daughter2 = a_D2[iParticle]
            daughter1_PID = a_PID[daughter1]
            daughter2_PID = a_PID[daughter2]
            #print('Event ',iEvt,'has higgs at position',iParticle,'with daughter1 (',daughter1,
            #    ') of PID',daughter1_PID,'and daughter2 (',daughter2,') of PID',daughter2_PID)
            if abs(daughter1_PID) == 5 and abs(daughter2_PID)==5:
                bQuarkIndices.append(daughter1)
                bQuarkIndices.append(daughter2)
    
    return bQuarkIndices


def returnListOfTruthBQuarkIndicesByStatus(a_status):
    bQuarkIndices = []

    for iParticle in range(0, len(a_status)):
        if a_status[iParticle]==23:
            bQuarkIndices.append(iParticle)

    return bQuarkIndices


In [42]:


def returnNumberAndListOfJetIndicesPassingCuts(a_jetPt, a_jetEta, cut_jetPt, cut_jetEta):
    _jetIndices = []
    _nJets = 0
    
    for iJet in range(0, len(a_jetPt)): 
        if a_jetPt[iJet] > cut_jetPt and abs(a_jetEta[iJet]) < cut_jetEta:
            _jetIndices.append(iJet)
            _nJets += 1
            
    return _nJets, _jetIndices

In [57]:
delphes_hh= uproot.open('../../../MG5_aMC_v2_6_1/ppToHHto4b_14TeV/Events/run_02_decayed_1/tag_1_delphes_events.root')['Delphes']
#b_particles = uproot.tree.TBranchMethods.array(delphes_hh['Particle'])
#l_genD1     = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.D1']).tolist()
#l_genD2     = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.D2']).tolist()
l_genPID    = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.PID']).tolist()
l_genStatus = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Status']).tolist()
l_genPt     = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.PT']).tolist()
l_genEta    = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Eta']).tolist()
l_genPhi    = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Phi']).tolist()
l_genMass   = uproot.tree.TBranchMethods.array(delphes_hh['Particle']['Particle.Mass']).tolist()
l_jetPt     = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.PT']).tolist()
l_jetEta    = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.Eta']).tolist()
l_jetPhi    = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.Phi']).tolist()
l_jetMass   = uproot.tree.TBranchMethods.array(delphes_hh['Jet']['Jet.Mass']).tolist()

nEventsWithExactly4Jets = 0
nEventsWithAtLeast4Jets = 0
nEventsWith4MatchableJets = 0

for iEvt in range(0,delphes_hh.fEntries):
    # *** 0. Kick-out condition for testing
    if iEvt > 40 and testRun is True:
        continue
    if iEvt%1000==0:
        print("Analyzing event number",iEvt)
    
    # *** 1. Get truth information
    indicesByStatus    = returnListOfTruthBQuarkIndicesByStatus(l_genStatus[iEvt])   
    if len(indicesByStatus) != 4:
        print ("!!! WARNING: iEvt = {0} did not find 4 truth b-quarks. Only found {1} !!!".format(iEvt, len(indicesByStatus)))
        continue
    #print(indicesByDaughters)
        
    # *** 2. Get jet reco information
    nJets, jetIndices = returnNumberAndListOfJetIndicesPassingCuts(l_jetPt[iEvt], l_jetEta[iEvt], 20.0, 2.5)
    if nJets < 4:
        continue
    nEventsWithAtLeast4Jets += 1
    #print(nJets, jetIndices)
    if nJets == 4:
        nEventsWithExactly4Jets += 1
    
    # *** 3. Do some matching
    matchedQuarksToJets = {}
    for iQuark in indicesByStatus:
        tlv_quark = TLorentzVector.PtEtaPhiMassLorentzVector( l_genPt[iEvt][iQuark], l_genEta[iEvt][iQuark], l_genPhi[iEvt][iQuark], l_genMass[iEvt][iQuark])
        for iJet in jetIndices:
            tlv_jet = TLorentzVector.PtEtaPhiMassLorentzVector( l_jetPt[iEvt][iJet], l_jetEta[iEvt][iJet], l_jetPhi[iEvt][iJet], l_jetMass[iEvt][iJet])

            # skip jets
            if tlv_quark.delta_r( tlv_jet) > dR_cut_quarkToJet:
                continue
                
            if iQuark not in matchedQuarksToJets.keys():
                matchedQuarksToJets.update({iQuark:[iJet]})
            else:
                matchedQuarksToJets[iQuark].append(iJet)

    #print( matchedQuarksToJets, all(len(matchedJets) == 1 for matchedJets in matchedQuarksToJets.values()) )
    if all(len(matchedJets) == 1 for matchedJets in matchedQuarksToJets.values()) and len(matchedQuarksToJets)==4:
        nEventsWith4MatchableJets += 1
        
print("Number of Events with >= 4 jet:",nEventsWithAtLeast4Jets)
print("Number of Events with == 4 jet:",nEventsWithExactly4Jets)
print("Number of Events with 4 truth-matchable jets:",nEventsWith4MatchableJets)
    


Analyzing event number 0
Analyzing event number 1000
Analyzing event number 2000
Analyzing event number 3000
Analyzing event number 4000
Analyzing event number 5000
Analyzing event number 6000
Analyzing event number 7000
Analyzing event number 8000
Analyzing event number 9000
Number of Events with >= 4 jet: 8164
Number of Events with == 4 jet: 2694
Number of Events with 4 truth-matchable jets: 532


In [58]:
a=TLorentzVector.PtEtaPhiMassLorentzVector(1,1,1,1)
a.E

1.8387761814701145